<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

# Python for Algorithmic Trading

**Chapter 09 &mdash; Stock Trading with Interactive Brokers**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

## A Wrapper Class for the IB API

In [ ]:
import tpqib

In [ ]:
con = tpqib.tpqibcon()

## Retrieving Historical Data

In [ ]:
aapl_contract = con.create_contract('AAPL', 'STK', 'SMART', 'SMART', 'USD')

In [ ]:
con.req_contract_details(aapl_contract)

In [ ]:
con.get_market_data_once(aapl_contract)

In [ ]:
import datetime as dt

In [ ]:
end_date = dt.datetime.now()

In [ ]:
request_id = con.request_historical_data(aapl_contract, end_date,
                                         '1 W', '8 hours', 'TRADES')

In [ ]:
request_id

In [ ]:
con.get_historical_data(request_id)

In [ ]:
request_id = con.request_historical_data(aapl_contract, end_date,
                                         '1 W', '1 min', 'TRADES')

In [ ]:
request_id

In [ ]:
data = con.get_historical_data(request_id)

In [ ]:
data.info()

In [ ]:
data.tail(10)

In [ ]:
data['WAP'].plot(figsize=(10, 6))
plt.savefig('../../images/ch09/ib_plot_01.png')

In [ ]:
tpqib.VALID_BAR_SIZES

## Working with Streaming Data from the IB API

In [ ]:
tpqib.STREAMING_DATA_TYPES

In [ ]:
def print_bid_ask_price(field, value):
    if field == 'bidPrice':
        print(dt.datetime.now(), '| bid is %s' % value)
    if field == 'askPrice':
        print(dt.datetime.now(), '| ask is %s' % value)

In [ ]:
request_id = con.request_market_data(aapl_contract,
                                     print_bid_ask_price)

In [ ]:
con.cancel_market_data(request_id)

## Implementing Trading Strategies in Real-Time

In [ ]:
import tpqib

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import datetime as dt

In [ ]:
con = tpqib.tpqibcon()

In [ ]:
symbol = 'AAPL'

In [ ]:
shares = 100

In [ ]:
contract = con.create_contract(symbol, 'STK', 'SMART', 'SMART', 'USD')

In [ ]:
buy_order = con.create_order('MKT', shares, 'Buy')

In [ ]:
sell_order = con.create_order('MKT', shares, 'Sell')

In [ ]:
con.place_order(contract, buy_order)

In [ ]:
con.place_order(contract, sell_order)

In [ ]:
def SMA_strategy(field, value):
        global data
        global ticks
        global resam
        global position
        global request_id
        if field == 'bidPrice':
            ticks += 1
            timestamp = dt.datetime.now()
            data = data.append(pd.DataFrame({'bid': value},
                                            index=[timestamp]))
            print('%3d | ' % ticks, timestamp, '| bid is %s' % value)
        elif field == 'askPrice':
            ticks += 1
            timestamp = dt.datetime.now()
            data = data.append(pd.DataFrame({'ask': value},
                                            index=[timestamp]))
            print('%3d | ' % ticks, timestamp, '| ask is %s' % value)
        resam = data.resample('1s').last().ffill()
        if len(resam) >= 10:
            resam['SMA1'] = resam.mean(axis=1).rolling(5).mean()
            resam['SMA2'] = resam.mean(axis=1).rolling(10).mean()
            resam['position'] = np.where(resam['SMA1'] > resam['SMA2'], 1, -1)
            if resam['position'].ix[-1] == 1 and position == 0:
                con.place_order(contract, buy_order)
                position = 1
            elif resam['position'].ix[-1] == -1 and position == 1:
                con.place_order(contract, sell_order)
                position = 0

In [ ]:
ticks = 0

In [ ]:
position = 0

In [ ]:
data = pd.DataFrame()

In [ ]:
request_id = con.request_market_data(contract, SMA_strategy)

In [ ]:
con.cancel_market_data(request_id)

In [ ]:
data.tail()

In [ ]:
resam.tail()

## Retrieving Account Information

In [ ]:
import tpqib

In [ ]:
con = tpqib.tpqibcon()

In [ ]:
symbols = ['AAPL', 'GOOG', 'YHOO']

In [ ]:
contracts = {sym: con.create_contract(sym, 'STK', 'SMART', 'SMART', 'USD')
             for sym in symbols}

In [ ]:
contracts

In [ ]:
buy_order = con.create_order('MKT', 50, 'Buy')

In [ ]:
for sym in symbols:
    con.place_order(contracts[sym], buy_order)

In [ ]:
con.req_positions()

In [ ]:
sell_order = con.create_order('MKT', 50, 'Sell')

In [ ]:
for sym in symbols:
    con.place_order(contracts[sym], sell_order)

In [ ]:
con.req_positions()

<img src="http://hilpisch.com/tpq_logo.png" alt="The Python Quants" width="35%" align="right" border="0"><br>

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

**Python Quant Platform** |
<a href="http://quant-platform.com">http://quant-platform.com</a>

**Python for Finance** |
<a href="http://python-for-finance.com" target="_blank">Python for Finance @ O'Reilly</a>

**Derivatives Analytics with Python** |
<a href="http://derivatives-analytics-with-python.com" target="_blank">Derivatives Analytics @ Wiley Finance</a>

**Listed Volatility and Variance Derivatives** |
<a href="http://lvvd.tpq.io" target="_blank">Listed VV Derivatives @ Wiley Finance</a>

**Python Training** |
<a href="http://training.tpq.io" target="_blank">Python for Finance University Certificate</a>